In [17]:
from botorch.models.transforms.input import InputStandardize
from botorch import fit_gpytorch_mll
import torch
from botorch.models import SingleTaskGP
from botorch.models.transforms import Normalize, Standardize, ChainedInputTransform
from gpytorch.mlls import ExactMarginalLogLikelihood

# Generate some example training data
train_X = torch.rand(10, 2, dtype=torch.double) + torch.rand(10, 2, dtype=torch.double) + torch.rand(10, 2, dtype=torch.double)  # 20 samples, 2 features
train_Y = torch.sin(train_X[:, 0]) + torch.cos(train_X[:, 1])  # Some function of the inputs
print(f"Y-mean -> {train_Y.mean()}")

# Define the input transformations
normalize = Normalize(d=train_X.shape[-1])
standardize = InputStandardize(d=train_X.shape[-1])
input_transform = normalize# ChainedInputTransform(tf1=standardize, tf2=normalize)
output_transform = Standardize(m=1)

print(f"Pre-transform X:\n{train_X}")
print(f"Mean -> {train_X.mean()}")
# Apply the transformations to the training data
transformed_X = input_transform(train_X)
print(f"Transformed X:\n{transformed_X}")
print(f"Mean -> {transformed_X.mean()}")

# Define and train the GP model
gp = SingleTaskGP(train_X, train_Y.unsqueeze(-1), input_transform=input_transform,
                  outcome_transform=output_transform)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)

# Fit the model
fit_gpytorch_mll(mll)

# Test the transformations during prediction
test_X = torch.rand(5, 2)  # 5 test samples
transformed_test_X = input_transform(test_X)
print(f"Transformed Test X:\n{transformed_test_X}")

# Make predictions
gp.eval()
with torch.no_grad():
    posterior = gp.posterior(test_X)
    mean = posterior.mean
    lower, upper = posterior.mvn.confidence_region()
    print(f"Predicted mean:\n{mean}")
    print(f"Confidence region:\n{lower}, {upper}")


Y-mean -> 1.0274299453993332
Pre-transform X:
tensor([[1.5768, 1.1933],
        [1.5130, 1.9399],
        [0.9817, 0.9119],
        [1.2342, 1.8790],
        [1.4635, 1.6702],
        [1.1987, 1.0060],
        [1.9782, 1.3785],
        [1.7853, 1.4926],
        [0.6445, 1.3008],
        [1.8866, 1.7320]], dtype=torch.float64)
Mean -> 1.4383391601648818
Transformed X:
tensor([[0.6990, 0.2737],
        [0.6512, 1.0000],
        [0.2529, 0.0000],
        [0.4421, 0.9408],
        [0.6141, 0.7377],
        [0.4156, 0.0916],
        [1.0000, 0.4540],
        [0.8554, 0.5649],
        [0.0000, 0.3783],
        [0.9313, 0.7978]], dtype=torch.float64)
Mean -> 0.5550238259229798
Transformed Test X:
tensor([[-0.3890, -0.5239],
        [-0.2514, -0.0018],
        [ 0.0415, -0.2632],
        [-0.1312, -0.2806],
        [ 0.0025, -0.7293]])
Predicted mean:
tensor([[1.1217],
        [1.0461],
        [1.3790],
        [1.2606],
        [1.2414]], dtype=torch.float64)
Confidence region:
tensor([0.532

In [18]:
import torch
from botorch import fit_gpytorch_mll
from botorch.models import MixedSingleTaskGP
from botorch.models.transforms import Normalize, Standardize, ChainedInputTransform
from botorch.models.transforms.input import InputStandardize
from gpytorch.mlls import ExactMarginalLogLikelihood

# Generate some example training data
train_X = torch.rand(10, 2, dtype=torch.double) + torch.rand(10, 2, dtype=torch.double) + torch.rand(10, 2, dtype=torch.double)  # 10 samples, 2 features
train_Y = torch.sin(train_X[:, 0]) + torch.cos(train_X[:, 1])  # Some function of the inputs
print(f"Y-mean -> {train_Y.mean()}")

# Define the input transformations
normalize = Normalize(d=train_X.shape[-1])
standardize = InputStandardize(d=train_X.shape[-1])
input_transform = ChainedInputTransform(tf1=standardize, tf2=normalize)
output_transform = Standardize(m=1)

print(f"Pre-transform X:\n{train_X}")
print(f"Mean -> {train_X.mean()}")
# Apply the transformations to the training data
transformed_X = input_transform(train_X)
print(f"Transformed X:\n{transformed_X}")
print(f"Mean -> {transformed_X.mean()}")

# Define and train the GP model
# Assuming the first dimension is categorical and the second is continuous
cat_dims = [0]  # indices of the categorical dimensions
gp = MixedSingleTaskGP(train_X, train_Y.unsqueeze(-1), cat_dims=cat_dims, 
                       input_transform=input_transform, outcome_transform=output_transform)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)

# Fit the model
fit_gpytorch_mll(mll)

# Test the transformations during prediction
test_X = torch.rand(5, 2, dtype=torch.double)  # 5 test samples
transformed_test_X = input_transform(test_X)
print(f"Transformed Test X:\n{transformed_test_X}")

# Make predictions
gp.eval()
with torch.no_grad():
    posterior = gp.posterior(test_X)
    mean = posterior.mean
    lower, upper = posterior.mvn.confidence_region()
    print(f"Predicted mean:\n{mean}")
    print(f"Confidence region:\n{lower}, {upper}")


Y-mean -> 1.2979449091156885
Pre-transform X:
tensor([[0.8560, 1.4173],
        [1.9688, 0.8463],
        [1.7538, 1.3478],
        [1.3922, 0.5609],
        [1.8664, 1.7617],
        [1.3709, 0.6106],
        [1.4558, 1.3360],
        [1.5754, 1.0900],
        [1.0111, 1.1575],
        [0.9741, 1.4501]], dtype=torch.float64)
Mean -> 1.2901350348098481
Transformed X:
tensor([[0.0000, 0.7132],
        [1.0000, 0.2376],
        [0.8068, 0.6553],
        [0.4819, 0.0000],
        [0.9080, 1.0000],
        [0.4627, 0.0414],
        [0.5390, 0.6455],
        [0.6465, 0.4406],
        [0.1394, 0.4968],
        [0.1062, 0.7404]], dtype=torch.float64)
Mean -> 0.503065986366696
Transformed Test X:
tensor([[-0.7547,  0.0783],
        [-0.4770, -0.3840],
        [-0.6239,  0.3345],
        [ 0.0415,  0.3118],
        [-0.7068, -0.4434]], dtype=torch.float64)
Predicted mean:
tensor([[1.7604],
        [1.9487],
        [1.5126],
        [1.5379],
        [1.9506]], dtype=torch.float64)
Confidence r

In [19]:
# Define and train the GP model
# Assuming the first dimension is categorical and the second is continuous
cat_dims = [0]  # indices of the categorical dimensions
gp = MixedSingleTaskGP(train_X, train_Y.unsqueeze(-1), cat_dims=cat_dims, 
                       input_transform=input_transform, outcome_transform=output_transform)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)

# Fit the model
fit_gpytorch_mll(mll)

# Test the transformations during prediction
test_X = torch.rand(5, 2, dtype=torch.double)  # 5 test samples
transformed_test_X = input_transform(test_X)
print(f"Transformed Test X:\n{transformed_test_X}")

# Make predictions
gp.eval()
with torch.no_grad():
    posterior = gp.posterior(test_X)
    mean = posterior.mean
    lower, upper = posterior.mvn.confidence_region()
    print(f"Predicted mean:\n{mean}")
    print(f"Confidence region:\n{lower}, {upper}")

Transformed Test X:
tensor([[-0.2403,  0.2990],
        [-0.0699, -0.2115],
        [-0.0565, -0.1867],
        [-0.4090, -0.4091],
        [-0.1857, -0.1123]], dtype=torch.float64)
Predicted mean:
tensor([[1.5520],
        [1.9172],
        [1.9092],
        [1.9500],
        [1.8792]], dtype=torch.float64)
Confidence region:
tensor([1.3536, 1.5968, 1.6072, 1.4537, 1.6243], dtype=torch.float64), tensor([1.7505, 2.2377, 2.2113, 2.4463, 2.1341], dtype=torch.float64)


# Attempt to replicate scenario

In [20]:
import torch
from botorch import fit_gpytorch_mll
from botorch.models import MixedSingleTaskGP
from botorch.models.transforms import Normalize, Standardize, ChainedInputTransform
from botorch.models.transforms.input import InputStandardize
from gpytorch.mlls import ExactMarginalLogLikelihood

# Generate some example training data
# 10 samples, 3 features (2 continuous and 1 discrete)
continuous_features = torch.rand(10, 2, dtype=torch.double)
discrete_feature = torch.randint(0, 3, (10, 1), dtype=torch.double)  # Discrete feature with 3 categories
train_X = torch.cat([discrete_feature, continuous_features], dim=1)
train_Y = torch.sin(train_X[:, 1]) + torch.cos(train_X[:, 2])  # Some function of the continuous inputs
print(f"Y-mean -> {train_Y.mean()}")

# Define the input transformations
input_transform = Normalize(d=train_X.shape[-1])
output_transform = Standardize(m=1)

print(f"Pre-transform X:\n{train_X}")
print(f"Mean -> {train_X.mean()}")

# Apply the transformations to the training data
transformed_X = input_transform(train_X)
print(f"Transformed X:\n{transformed_X}")
print(f"Mean -> {transformed_X.mean()}")

# Define and train the GP model
# Assuming the first dimension is categorical and the next two are continuous
cat_dims = [0]  # indices of the categorical dimensions
gp = MixedSingleTaskGP(train_X, train_Y.unsqueeze(-1), cat_dims=cat_dims, 
                       input_transform=input_transform, outcome_transform=output_transform)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)

# Fit the model
fit_gpytorch_mll(mll)

# Test the transformations during prediction
# Generate test data with 5 samples
continuous_features_test = torch.rand(5, 2, dtype=torch.double)
discrete_feature_test = torch.randint(0, 3, (5, 1), dtype=torch.double)  # Discrete feature with 3 categories
test_X = torch.cat([discrete_feature_test, continuous_features_test], dim=1)
transformed_test_X = input_transform(test_X)
print(f"Transformed Test X:\n{transformed_test_X}")

# Make predictions
gp.eval()
with torch.no_grad():
    posterior = gp.posterior(test_X)
    mean = posterior.mean
    lower, upper = posterior.mvn.confidence_region()
    print(f"Predicted mean:\n{mean}")
    print(f"Confidence region:\n{lower}, {upper}")


Y-mean -> 1.3763063820719779
Pre-transform X:
tensor([[1.0000, 0.8475, 0.4174],
        [0.0000, 0.5489, 0.6960],
        [2.0000, 0.7019, 0.6020],
        [0.0000, 0.9936, 0.4755],
        [2.0000, 0.1989, 0.1537],
        [1.0000, 0.5213, 0.8255],
        [2.0000, 0.7160, 0.0482],
        [1.0000, 0.0907, 0.4729],
        [0.0000, 0.3687, 0.4942],
        [0.0000, 0.4802, 0.4210]], dtype=torch.float64)
Mean -> 0.6357978286383106
Transformed X:
tensor([[0.5000, 0.8382, 0.4750],
        [0.0000, 0.5074, 0.8335],
        [1.0000, 0.6769, 0.7125],
        [0.0000, 1.0000, 0.5498],
        [1.0000, 0.1198, 0.1356],
        [0.5000, 0.4769, 1.0000],
        [1.0000, 0.6926, 0.0000],
        [0.5000, 0.0000, 0.5464],
        [0.0000, 0.3079, 0.5738],
        [0.0000, 0.4314, 0.4796]], dtype=torch.float64)
Mean -> 0.49524299745423667
Transformed Test X:
tensor([[1.0000, 0.0482, 0.7040],
        [1.0000, 0.4506, 0.2670],
        [1.0000, 0.2146, 0.0394],
        [1.0000, 0.5226, 0.1680],
    